In [1]:
import itertools
import os
import sys
from pathlib import Path

import cv2
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import skimage.io

from collections import defaultdict
from tqdm.auto import tqdm
from joblib import Parallel, delayed
import re
import h5py
import napari
import seaborn as sns
from sklearn.preprocessing import StandardScaler


In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
p_dir = (Path().cwd().parents[0]).absolute()

module_path = str(p_dir / "src")

if module_path not in sys.path:
    sys.path.append(module_path)

In [4]:
data_dir = (Path().cwd().parents[0] / 'data').absolute()

In [5]:
import PPIGraph

# Read per cell data

In [6]:
from torch_geometric.loader import DataLoader

In [7]:
# Define condition mapping
condition_mapping = {'1W': 0, '1M': 1}

# Load graph dataset and process if neede
graph_path = data_dir / 'OCT mouse' / 'graphs' 

In [8]:
# Filter out by maximum number of counts per cell
min_count = 5
max_count = 40

dataset = PPIGraph.GraphDataset(graph_path, 'raw', 'pt',  condition_mapping=condition_mapping, n_c=2)

# Create Dataloader
loader = DataLoader(dataset, batch_size=1, shuffle=False)

# # Get Indices
# indices = []
# for step, data in enumerate(loader):
#     if len(data.x) <= min_count:
#         continue 
    
#     if (data.x.sum(axis=0) >= max_count).any():
#         continue
#     indices.append(step)
    
# # Get subset dataset
# dataset_filtered = dataset.index_select(indices)


In [9]:
# np.save(graph_path / 'indices.npy', indices)
# indices_test = np.load(graph_path / 'indices.npy')

# np.array_equal(indices_test,np.array(indices))

In [10]:
dataset

GraphDataset(49629)

In [11]:
indices = np.load(graph_path / 'indices.npy')
indices = indices[::15]
dataset_filtered = dataset.index_select(list(indices))


In [12]:
dataset_filtered

GraphDataset(2901)

In [13]:
dataset_filtered[0]

Data(edge_index=[2, 100], pos=[20, 2], labels=[20, 5], nuclei=[20], weight=[100], condition=0, fov=1, id=100, train_mask=[20], test_mask=[20], edge_attr=[100, 2], x=[20, 5], y=[1], edge_weight=[100], name='0_1_100.gpt')

# ML model

In [14]:
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn import preprocessing, metrics
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression

from sklearn.model_selection import train_test_split, KFold
from sklearn.utils.class_weight import compute_class_weight
import wandb

In [15]:
# condition = 'ML'
# project_name = f'PLA_xeno_11324_{condition}'

In [16]:
# models = {
#     'Adaboost': AdaBoostClassifier(),
#     'DecisionTree': DecisionTreeClassifier(),
#     'GradientBoosting' : GradientBoostingClassifier(),
#     'NaiveBayes': GaussianNB(),
#     'RandomForest': RandomForestClassifier(), 
#     'SVM': SVC(probability =True),
#     'LogisticRegression':  LogisticRegression(),
#     'MLP': MLPClassifier(random_state=1, max_iter=100, hidden_layer_sizes=[16, 16, 16])
# }

In [17]:
# import torch 

# data = torch.empty((0, 5))
# label = []

# for graph in dataset_filtered:
#     new_row =graph.x.sum(axis = 0).unsqueeze(0)
#     data = torch.cat((data, new_row), dim=0)
#     label.append(graph.condition)


In [18]:
# np.save(graph_path / 'data.npy', data)
# np.save(graph_path / 'label.npy', label)

In [19]:
# scaler = StandardScaler()

# # Run model on cell count
# X = data.numpy()
# X = scaler.fit_transform(X)
# y = np.array(label)

# # X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=0)
# kfold = KFold(n_splits = 10, shuffle = True, random_state = 0)

# for model_name, model in models.items():
#     for k, (train_index, test_index) in enumerate(kfold.split(X)):
#         # Split the dataset
#         X_train, X_test = X[train_index], X[test_index]
#         y_train, y_test = y[train_index], y[test_index]
        
#         model.fit(X_train, y_train)
#         y_pred = model.predict(X_test)
#         y_probas = model.predict_proba(X_test)

#         run = wandb.init(project=project_name, group=model_name+'_cell', name=model_name+f'_cell_{k}')

#         accuracy = metrics.accuracy_score(y_test, y_pred)
#         b_accuracy = metrics.balanced_accuracy_score(y_test, y_pred)
#         f1 = metrics.f1_score(y_test, y_pred)
#         auc = metrics.roc_auc_score(y_test, y_pred)
#         wandb.log({"accuracy": accuracy, 'b_accuracy': b_accuracy, 'f1':f1, 'auc': auc})
#     run.finish()

In [20]:
# import torch 

# data = torch.empty((0, 10))
# label = []

# for graph in dataset_filtered:
#     cell =graph.x.sum(axis = 0).unsqueeze(0)

#     nuclei = torch.tile(graph.nuclei, (5, 1)).T * graph.x
#     nuclei = nuclei.sum(axis=0).unsqueeze(0)
#     cyto = cell - nuclei
#     new_row = torch.cat([cyto, nuclei], axis=1)
#     data = torch.cat((data, new_row), dim=0)
#     label.append(graph.condition)

In [21]:
# scaler = StandardScaler()

# # Run model on cell count
# X = data.numpy()
# X = scaler.fit_transform(X)
# y = np.array(label)

# # X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=0)
# kfold = KFold(n_splits = 5, shuffle = True, random_state = 0)

# for model_name, model in models.items():
#     for k, (train_index, test_index) in enumerate(kfold.split(X)):
#         # Split the dataset
#         X_train, X_test = X[train_index], X[test_index]
#         y_train, y_test = y[train_index], y[test_index]
        
#         model.fit(X_train, y_train)
#         y_pred = model.predict(X_test)
#         y_probas = model.predict_proba(X_test)

#         run = wandb.init(project=project_name, group=model_name+'_cytonuclei', name=model_name+f'_cytonuclei_{k}')

#         accuracy = metrics.accuracy_score(y_test, y_pred)
#         b_accuracy = metrics.balanced_accuracy_score(y_test, y_pred)
#         f1 = metrics.f1_score(y_test, y_pred)
#         auc = metrics.roc_auc_score(y_test, y_pred)
#         wandb.log({"accuracy": accuracy, 'b_accuracy': b_accuracy, 'f1':f1, 'auc': auc})
#     run.finish()

# GNN training

In [22]:
import lightning.pytorch as pl
import PPIGraph
from sklearn.model_selection import train_test_split, KFold


In [23]:
import itertools
from lightning.pytorch.accelerators import find_usable_cuda_devices
import wandb

In [24]:
dataset_name = 'xenograph'
condition = 'Kfold'
checkpoint_folder = (Path().cwd().parents[0]).absolute() / 'data' /'OCT mouse' /"saved_models" / dataset_name / f"Graph_GNNs_{condition}" 
project_name = f'PLA_01122024_{dataset_name}_{condition}'

In [25]:
AVAIL_GPUS = [1]

# Setting the seed
pl.seed_everything(42)
epochs = 20

# NUM_LAYERS = 2
# HIDDEN_CHANNELS = 16
# pools = ['mean', 'max', 'sum', 'attention', 'attention2']
# num_layers = [2,3,4]
# hiddens = [16, 32, 64]


# # model = 'GAT'
# model = 'GINConv'
params = [[3,32,'mean','MLP'], 
[3,32,'attention2','GraphConv'],
[2,64,'attention2','GCN'],
[3,64,'attention','GAT'],
[2,16,'attention','GINConv']]

Global seed set to 42


In [26]:
import torch
torch.set_float32_matmul_precision('medium')

In [27]:
k_folds = 5
kfold = KFold(n_splits=k_folds, shuffle=True, random_state=42)

for fold, (train_ids, valid_ids) in enumerate(kfold.split(dataset_filtered)):
    train_subset = dataset_filtered.index_select(train_ids.tolist())
    val_subset = dataset_filtered.index_select(valid_ids.tolist())
    for NUM_LAYERS, HIDDEN_CHANNELS, pool, model in params:
            # Path to the folder where the pretrained models are saved
        # CHECKPOINT_PATH = checkpoint_folder / f'GAT_{NUM_LAYERS}_{HIDDEN_CHANNELS}_onehot_{fold}' / pool
        CHECKPOINT_PATH = checkpoint_folder / f'{model}_{NUM_LAYERS}_{HIDDEN_CHANNELS}_onehot_{fold}' / pool
        CHECKPOINT_PATH.mkdir(parents=True, exist_ok=True)

        # Skip already trained kfold and pool
        checkpoint = CHECKPOINT_PATH / f"GraphLevel{model}" / f"GraphLevel{model}.ckpt" 
        if checkpoint.exists():
            print(checkpoint)
            continue

        # Run training
        run = wandb.init(project=project_name, name=f'{model}_{NUM_LAYERS}_{HIDDEN_CHANNELS}_onehot_{fold}', 
                        group=f'{model}_{pool}', 
                        # mode="disabled"
                        )
        PPIGraph.train_graph_classifier_kfold(model, 
                                                train_subset, 
                                                val_subset, 
                                                dataset_filtered, 
                                                CHECKPOINT_PATH, 
                                                AVAIL_GPUS, 
                                                in_channels=5,
                                                hidden_channels=HIDDEN_CHANNELS, 
                                                out_channels = HIDDEN_CHANNELS,
                                                num_layers=NUM_LAYERS, 
                                                epochs=epochs,
                                                embedding=False,
                                                batch_size=256,
                                                num_workers= 24,
                                                graph_pooling=pool)
        run.finish()


y:\coskun-lab\Thomas\23_PLA_revision\data\OCT mouse\saved_models\xenograph\Graph_GNNs_Kfold\MLP_3_32_onehot_0\mean\GraphLevelMLP\GraphLevelMLP.ckpt
y:\coskun-lab\Thomas\23_PLA_revision\data\OCT mouse\saved_models\xenograph\Graph_GNNs_Kfold\GraphConv_3_32_onehot_0\attention2\GraphLevelGraphConv\GraphLevelGraphConv.ckpt
y:\coskun-lab\Thomas\23_PLA_revision\data\OCT mouse\saved_models\xenograph\Graph_GNNs_Kfold\GCN_2_64_onehot_0\attention2\GraphLevelGCN\GraphLevelGCN.ckpt
y:\coskun-lab\Thomas\23_PLA_revision\data\OCT mouse\saved_models\xenograph\Graph_GNNs_Kfold\GAT_3_64_onehot_0\attention\GraphLevelGAT\GraphLevelGAT.ckpt
y:\coskun-lab\Thomas\23_PLA_revision\data\OCT mouse\saved_models\xenograph\Graph_GNNs_Kfold\GINConv_2_16_onehot_0\attention\GraphLevelGINConv\GraphLevelGINConv.ckpt
y:\coskun-lab\Thomas\23_PLA_revision\data\OCT mouse\saved_models\xenograph\Graph_GNNs_Kfold\MLP_3_32_onehot_1\mean\GraphLevelMLP\GraphLevelMLP.ckpt
y:\coskun-lab\Thomas\23_PLA_revision\data\OCT mouse\saved_mo

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: thoomas. Use `wandb login --relogin` to force relogin


Global seed set to 42
c:\Users\thu71\Anaconda3\envs\snowflake\lib\site-packages\pytorch_lightning\loggers\wandb.py:395: UserWarning: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
  rank_zero_warn(
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Global seed set to 42


Using GNN


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]

   | Name        | Type             | Params
--------------------------------------------------
0  | x_embedding | Identity         | 0     
1  | model       | GNNModel         | 4.6 K 
2  | head        | Sequential       | 562   
3  | loss_module | CrossEntropyLoss | 0     
4  | train_acc   | BinaryAccuracy   | 0     
5  | train_auroc | BinaryAUROC      | 0     
6  | train_f1    | BinaryF1Score    | 0     
7  | valid_acc   | BinaryAccuracy   | 0     
8  | valid_auroc | BinaryAUROC      | 0     
9  | valid_f1    | BinaryF1Score    | 0     
10 | pool        | Attention_module | 1.1 K 
--------------------------------------------------
6.3 K     Trainable params
0         Non-trainable params
6.3 K     Total params
0.025     Total estimated model params size (MB)
C:\Users\thu71\AppData\Roaming\Python\Python39\site-packages\lightning\pytorch\utilities\data.py:77: UserWarning: Trying to infer the `batch_size` from an ambiguous collection. The ba

...

epoch,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▇▇▇▇▇▇████
lr-Adam,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_acc,▁▆▇▇▇▇█▇███▇████████
train_auc,▁▆▇▇█▇▇███████████▇█
train_f1,▁▆▇▇█▇█▇███████▇████
train_loss_epoch,██▄▆▂▃▃▂▂▂▂▃▁▂▁▁▂▁▂▂
train_loss_step,▂█▄▇▂▄▅▂▂▂▃▃▁▂▃▁▃▂▃▃
trainer/global_step,▁▁▁▁▂▁▂▁▃▁▃▂▃▃▃▂▄▄▄▂▅▅▅▂▆▆▂▂▆▆▃▃▇▇▃▃██▃█
val_acc,▁▄▄▅▇▇▇▇█▇▇▇▇▇▇▇█▇█▇
val_auc,▁▁▅▆▆▅▅▆▆▆▇█▇█▇▇█▄█▆
val_f1,▁▅▅▅▇▆▆▇▇▇▇▇▇▇▇▇█▇▇▇


Global seed set to 42
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Global seed set to 42
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]

   | Name        | Type             | Params
--------------------------------------------------
0  | x_embedding | Identity         | 0     
1  | model       | GIN              | 1.0 K 
2  | head        | Sequential       | 154   
3  | loss_module | CrossEntropyLoss | 0     
4  | train_acc   | BinaryAccuracy   | 0     
5  | train_auroc | BinaryAUROC      | 0     
6  | train_f1    | BinaryF1Score    | 0     
7  | valid_acc   | BinaryAccuracy   | 0     
8  | valid_auroc | BinaryAUROC      | 0     
9  | valid_f1    | BinaryF1Score    | 0     
10 | pool        | GlobalAttention  | 17    
--------------------------------------------------
1.2 K     Trainable params
0         Non-trainable params
1.2 K     Total params
0.005     Total estimated model para

GINConv


`Trainer.fit` stopped: `max_epochs=20` reached.


epoch,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▇▇▇▇▇▇████
lr-Adam,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_acc,▁▆▇▇▇▇▇██████████▇██
train_auc,▁▆▇█▇███████████████
train_f1,▁▆▇█▇█▇█████████▇███
train_loss_epoch,█▅▄▄▄▂▂▁▃▁▂▄▂▂▁▃▂▂▁▂
train_loss_step,▃▄▄▅▄▁▂▂▆▁▃█▄▃▂▇▂▂▂▅
trainer/global_step,▁▁▁▁▂▁▂▁▃▁▃▂▃▃▃▂▄▄▄▂▅▅▅▂▆▆▂▂▆▆▃▃▇▇▃▃██▃█
val_acc,▁▇▇▇▇▇█▇▇▇▇▇█▇▇▇█▇██
val_auc,▁▅▇▇▇▇▇▅▆▆▆▄▇▇▇▇█▇█▇
val_f1,▁▆▅▇▆▇▇▅▅▅▇▅▇▇▇▆█▇█▇
